# Twitter User Search
Get timeline data from twitter based on usernames

Requires enviornment variables:

- TWITTER_CONSUMER_KEY
- TWITTER_CONSUMER_SECRET
- TWITTER_APPLICATION_TOKEN
- TWITTER_APPLICATION_SECRET

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

from phoenix.scrape import twitter_queries
from phoenix.scrape import twitter_utilities
from phoenix.scrape import scrape
from phoenix.scrape import group_by
from phoenix.common import run_datetime
from phoenix.common import artifacts
from phoenix.common import utils

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
RUN_DATETIME = None

ARTIFACTS_ENVIRONMENT_KEY = "local"
ARTIFACT_SOURCE_TWEETS_URL = None

#The type of query: "user" or "keyword"
QUERY_TYPE = "user"
QUERIES_CSV_PATH = f"{artifacts.urls.get_static_config()}twitter_query_users.csv"
# The query: list of users for "user" query and list of keyword queries for "keyword"
# Number of days back to scrape
SINCE_DAYS = 3
# Total number of items, 0 for infinite
NUM_ITEMS = 0

In [ ]:
# DEFAULTS
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
aur = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY)
# Set Artefacts URL
if ARTIFACT_SOURCE_TWEETS_URL is None:
    ARTIFACT_SOURCE_TWEETS_URL = aur.get_url("source-user_tweets")

In [ ]:
print(
ARTIFACT_SOURCE_TWEETS_URL,
ARTIFACT_BASE_TWEETS_URL,
run_dt.dt,
QUERY_TYPE,
QUERIES_CSV_PATH,
SINCE_DAYS,
NUM_ITEMS,
sep='\n')

In [ ]:
queries = twitter_utilities.load_queries_from_csv(QUERIES_CSV_PATH)
print(queries)

In [ ]:
tweets_json=twitter_queries.get_tweets_json(QUERY_TYPE, queries, NUM_ITEMS, SINCE_DAYS)
len(tweets_json)

In [ ]:
a = artifacts.json.persist(ARTIFACT_SOURCE_TWEETS_URL, tweets_json);
a.url

In [ ]:
all_persisted = group_by.persist_tweets(aur, "user", tweets_json, run_dt, SINCE_DAYS)
for a in all_persisted:
    print(a.url)